### Deploy Workspace Monitoring Templates via Notebook

Run this notebook in the workspace, where the Workspace Monitoring feature has been already enabled.

The following items will be deployed:
- PBI Report + Semantic model
- RTI-Dashboard

Set the credential on the `Fabric Workspace Monitoring SM.SemanticModel` after deployment

### Deployment logic

In [ ]:
%pip install ms-fabric-cli --quiet

In [15]:
import subprocess
import os
import json
from zipfile import ZipFile 
import shutil
import re
import requests
import zipfile
from io import BytesIO
import yaml
import sempy.fabric as fabric

In [16]:
def download_folder_as_zip(repo_owner, repo_name, output_zip, branch="main", folder_to_extract="src",  remove_folder_prefix = ""):
    # Construct the URL for the GitHub API to download the repository as a zip file
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/zipball/{branch}"
    
    # Make a request to the GitHub API
    response = requests.get(url)
    response.raise_for_status()
    
    # Ensure the directory for the output zip file exists
    os.makedirs(os.path.dirname(output_zip), exist_ok=True)
    
    # Create a zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zipf:
        with zipfile.ZipFile(output_zip, 'w') as output_zipf:
            for file_info in zipf.infolist():
                parts = file_info.filename.split('/')
                if  re.sub(r'^.*?/', '/', file_info.filename).startswith(folder_to_extract): 
                    # Extract only the specified folder
                    file_data = zipf.read(file_info.filename)
                    output_zipf.writestr(('/'.join(parts[1:]).replace(remove_folder_prefix, "")), file_data)

def uncompress_zip_to_folder(zip_path, extract_to):
    # Ensure the directory for extraction exists
    os.makedirs(extract_to, exist_ok=True)
    
    # Uncompress all files from the zip into the specified folder
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Delete the original zip file
    os.remove(zip_path)

repo_owner = "Microsoft"
repo_name = "fabric-toolbox"
branch = "FWM-2025.8.2-3"
folder_prefix = "monitoring/workspace-monitoring-dashboards"

download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/src/src.zip", branch = branch, folder_to_extract= f"/{folder_prefix}/src", remove_folder_prefix = f"{folder_prefix}/")

In [18]:
# Definition of deployment functions

# Set environment parameters for Fabric CLI
token = notebookutils.credentials.getToken('pbi')
os.environ['FAB_TOKEN'] = token
os.environ['FAB_TOKEN_ONELAKE'] = token

def run_fab_command( command, capture_output: bool = False, silently_continue: bool = False):
    result = subprocess.run(["fab", "-c", command], capture_output=capture_output, text=True)
    if (not(silently_continue) and (result.returncode > 0 or result.stderr)):
       raise Exception(f"Error running fab command. exit_code: '{result.returncode}'; stderr: '{result.stderr}'")    
    if (capture_output): 
        output = result.stdout.strip()
        return output

def copy_to_tmp(name):
    shutil.rmtree("./builtin/tmp",  ignore_errors=True)
    path2zip = "./builtin/src/src.zip"
    with  ZipFile(path2zip) as archive:
        for file in archive.namelist():
            if file.startswith(f'src/{name}/'):
                archive.extract(file, './builtin/tmp')
    return(f"./builtin/tmp/src/{name}" )


In [19]:
# Get current Workspace

trg_workspace_id = fabric.get_notebook_workspace_id()
res = run_fab_command(f"api -X get workspaces/{trg_workspace_id}" , capture_output = True, silently_continue=True)
trg_workspace_name = json.loads(res)["text"]["displayName"]

print(f"Current workspace: {trg_workspace_name}")
print(f"Current workspace ID: {trg_workspace_id}")

Current workspace: FWM-test-01
Current workspace ID: 5f026589-af8e-4b87-8075-ba24f694f420


In [20]:
def fab_get_id(name):
    id = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name} -q id" , capture_output = True, silently_continue= True)
    return(id)

def fab_get_eventhouse_properties(name):
    id = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name} -q id" , capture_output = True, silently_continue= True)
    return(id)


In [ ]:
eh_id = fab_get_eventhouse_properties('Monitoring Eventhouse.Eventhouse')

In [ ]:
eventhouse_item = run_fab_command(f"api -X get workspaces/{trg_workspace_id}/eventhouses/{eh_id}" , capture_output = True, silently_continue=True)

In [ ]:
eventhouse_item_json = json.loads(eventhouse_item)
eventhouse_query_id = eventhouse_item_json['text']['properties']['queryServiceUri']
print(eventhouse_query_id)

In [24]:
# Define the deployment order for the resources
deployment_order = [
    {
        "name": "Fabric Workspace Monitoring SM.SemanticModel"
    },
    {
        "name": "Fabric Workspace Monitoring Report.Report"
    },
    {
        "name": "Fabric Workspace Monitoring Dashboard.KQLDashboard"
    }
]

In [25]:
def update_report_definition(path, workspace_name, semantic_model_id): 
    
    definition_path = os.path.join(path, "definition.pbir")
   
    with open(definition_path, "r", encoding="utf8") as file:
        report_definition = json.load(file)

    report_definition["datasetReference"]["byPath"] = None

    value = f'Data Source=\"powerbi://api.powerbi.com/v1.0/myorg/{workspace_name}\";initial catalog=\"Fabric Workspace Monitoring SM\";integrated security=ClaimsToken;semanticmodelid={semantic_model_id}'
    by_connection_obj = {
                "connectionString": value
            }

    report_definition["datasetReference"]["byConnection"] = by_connection_obj

    with open(definition_path, "w") as file:
            json.dump(report_definition, file, indent=4)

In [28]:
def replace_ids_in_folder(folder_path, query_uri):
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('expressions.tmdl'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    content = content.replace("CUSTOM_MONITORING_QUERY_URI", query_uri)
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(content)

In [ ]:
for it in deployment_order:

    name = it["name"]
    
    print("")
    print("#############################################")
    print(f"Deploying {name}")

    # Copy and replace IDs in the item
    tmp_path = copy_to_tmp(name)

    cli_parameter = ''

    if "Report" in name:
        time.sleep(10)
        trg_semantic_model_id = fab_get_id('Fabric Workspace Monitoring SM.SemanticModel')
        print(trg_semantic_model_id)
        time.sleep(10)
        print("INFO: Rebind Report to current semantic model")
        update_report_definition(tmp_path, trg_workspace_name, trg_semantic_model_id)
    if "SemanticModel" in name:
        print("INFO: Replace parameters for semantic model")
        print(eventhouse_query_id)
        replace_ids_in_folder(tmp_path, eventhouse_query_id)
    
    # Deploy items with mapped Ids
    run_fab_command(f"import  /{trg_workspace_name}.Workspace/{name} -i {tmp_path} -f {cli_parameter} ", silently_continue= True)